In [7]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import warnings

from utils import make_dataset

warnings.filterwarnings('ignore')


TARGET_COLS = 'two_year_recid'
COMPAS_SCORES_COLS = 'decile_score'
NUMERICAL_FEATURE_COLS = ['age',
                          'juv_fel_count','juv_misd_count','juv_other_count',
                          'priors_count','jail_time']
CATEGORICAL_FEATURE_COLS = ['sex','race',
                            'c_charge_degree']
FEATURE_NAMES = NUMERICAL_FEATURE_COLS+CATEGORICAL_FEATURE_COLS

PROTECTED_COLS = ['sex','race']

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')
data.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,jail_time,sex,race,c_charge_degree,decile_score,two_year_recid,compas_score
0,69,0,0,0,0,0.0,1,1,1,1,0,0.1
1,34,0,0,0,0,10.0,1,2,1,3,1,0.3
2,24,0,0,1,4,1.0,1,2,1,4,1,0.4
3,23,0,1,0,1,0.0,1,2,1,8,0,0.8
4,43,0,0,0,2,0.0,1,1,1,1,0,0.1


In [8]:
data['compas_class'] = (data['decile_score']<6).astype(int)

In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=1234)

In [10]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(test[TARGET_COLS], test['compas_score'])
roc_auc

0.7012452309472434

# Model fairness logger 

In [13]:
import neptune
from neptunecontrib.monitoring.fairness import log_fairness_classification_metrics

neptune.init('shared/model-fairness-workshop')
neptune.create_experiment(name='compas', tags=['debug', 'race'], upload_source_files=['*.py','*.ipynb'])

neptune.log_metric('roc_auc',roc_auc)
log_fairness_classification_metrics(test['two_year_recid'], test['compas_class'], test[['race']],
                                    favorable_label=0, unfavorable_label=1,
                                    privileged_groups = {'race':[3]}, unprivileged_groups={'race':[1,2,4,5,6]})

neptune.stop()

https://ui.neptune.ml/shared/model-fairness-workshop/e/MOD-2
